# 🥋 Lekcja 35: Weight Decay vs L2 Regularization (Adam vs AdamW)

Chcemy zapobiec overfittingowi, karając model za duże wagi.
Mamy dwa sposoby matematyczne:

1.  **L2 Regularization:** Dodajemy karę do funkcji kosztu:
    $$ Loss = MSE + \frac{\lambda}{2} ||w||^2 $$
    Gradient: $\nabla Loss = \nabla MSE + \lambda w$

2.  **Weight Decay:** Modyfikujemy regułę aktualizacji wagi (bez ruszania Loss):
    $$ w_{new} = w_{old} - lr \cdot \nabla MSE - lr \cdot \lambda \cdot w_{old} $$

**Wielki Sekret:**
*   Dla **SGD**, te dwie metody są matematycznie równoważne.
*   Dla **Adam**, NIE SĄ. Adam dzieli gradient przez "wariancję". Jeśli dodasz L2 do gradientu, Adam to "znormalizuje" i kara przestanie działać tak jak chcemy.
*   Dlatego powstał **AdamW** (Adam with decoupled Weight Decay), który stosuje metodę nr 2.

Sprawdzimy to eksperymentalnie.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from typing import Type

# Stała inicjalizacyjna
W_INIT_VALUE = 10.0

def run_experiment(optimizer_cls: Type[torch.optim.Optimizer], steps: int = 100, **optimizer_kwargs) -> float:
    """
    Symuluje pętlę treningową w izolacji, aby zbadać wpływ mechanizmów 
    wewnętrznych optymalizatora (np. Weight Decay) przy braku gradientu z danych.
    
    Args:
        optimizer_cls: Klasa optymalizatora (np. optim.SGD, optim.AdamW)
        steps: Liczba kroków symulacji
        **optimizer_kwargs: Parametry przekazywane do konstruktora (np. lr, weight_decay)
        
    Returns:
        float: Końcowa wartość wagi po optymalizacji.
    """
    # Inicjalizacja wagi (Leaf Tensor)
    w = torch.tensor([W_INIT_VALUE], requires_grad=True)
    
    # Inicjalizacja optymalizatora
    optimizer = optimizer_cls([w], **optimizer_kwargs)
    
    for _ in range(steps):
        optimizer.zero_grad()
        
        # --- MECHANIZM SZTUCZNEGO GRAFU ---
        # Tworzymy stratę zależną od 'w', ale mnożymy ją przez 0.
        # Efekt matematyczny: Loss = 0, Gradient z danych (dLoss/dw) = 0.
        # Cel: Zmuszenie PyTorcha do zbudowania grafu i uruchomienia .backward(),
        # co pozwoli optymalizatorowi zaaplikować Weight Decay.
        loss = w.sum() * 0.0
        
        loss.backward()
        optimizer.step()
        
    return w.item()

## Eksperyment 1: SGD

Dla SGD, parametr `weight_decay` w PyTorch implementuje matematykę L2.
Waga powinna maleć wykładniczo (Decay).

In [5]:
# SGD bez weight decay (powinno zostać 10.0)
res_sgd_none = run_experiment(optim.SGD, lr=0.1, weight_decay=0.0)

# SGD z weight decay
res_sgd_wd = run_experiment(optim.SGD, lr=0.1, weight_decay=0.1)

print(f"SGD (Bez WD): {res_sgd_none:.4f}")
print(f"SGD (Z WD):   {res_sgd_wd:.4f}")

if res_sgd_wd < res_sgd_none:
    print("✅ SGD poprawnie zmniejszyło wagę.")

SGD (Bez WD): 10.0000
SGD (Z WD):   3.6603
✅ SGD poprawnie zmniejszyło wagę.


## Eksperyment 2: Adam vs AdamW

Tutaj leży pies pogrzebany.
*   **`optim.Adam`**: Traktuje `weight_decay` jako L2 Regularization (dodaje do gradientu). Ponieważ Adam dzieli przez historię gradientów, ta kara L2 jest "wygładzana" i traci swoją siłę przy rzadkich cechach.
*   **`optim.AdamW`**: Odejmuje decay bezpośrednio od wagi, omijając mechanizm adaptacyjny Adama.

Przyjrzyjmy się wynikom. AdamW powinien zmniejszyć wagę **mocniej/bardziej przewidywalnie** niż Adam przy tych samych ustawieniach, w sytuacji gdy gradient z danych jest zerowy.

In [6]:
# Adam z 'weight_decay' (To jest implementacja L2!)
res_adam = run_experiment(optim.Adam, lr=0.1, weight_decay=0.1)

# AdamW z 'weight_decay' (To jest prawdziwe Decoupled Weight Decay)
res_adamw = run_experiment(optim.AdamW, lr=0.1, weight_decay=0.1)

print(f"Adam  (L2 style):    {res_adam:.4f}")
print(f"AdamW (Decoupled):   {res_adamw:.4f}")

# Analiza różnicy
diff = abs(res_adam - res_adamw)
print(f"\nRóżnica w wyniku: {diff:.4f}")

if res_adamw < res_adam:
    print("👉 AdamW zredukował wagę mocniej.")
    print("Dlaczego? W zwykłym Adamie, gdy gradient=0, mechanizm adaptacyjny 'dzielenia przez zero' (epsilon) zaburza karę L2.")
    print("AdamW aplikuje karę czysto matematycznie: w = w - lr * lambda * w.")

Adam  (L2 style):    2.2445
AdamW (Decoupled):   3.6603

Różnica w wyniku: 1.4159


## 🥋 Black Belt Summary

To jest najczęstszy błąd w implementacji Transformerów.
Wielu inżynierów używa `optim.Adam(model.parameters(), weight_decay=0.01)`.
To jest **BŁĄD**. To nie działa tak, jak myślisz.

**Złota Zasada:**
1.  Używasz SGD? -> `optim.SGD` (weight_decay działa ok).
2.  Używasz Adama? -> **ZAWSZE używaj `optim.AdamW`**.
3.  Zwykły `optim.Adam` używaj tylko wtedy, gdy `weight_decay=0` (bez regularyzacji).